In [89]:
# Import dependencies
import pandas as pd
import os
# import csv

## Import The Data

In [90]:
# Specify Filepaths
locations_inpath = "Resources/CitiesWGeolocation.csv"
cost_analysis_inpath = "Resources/cost_analysis.csv"
weather_inpath = "Resources/weather.csv"
# yelp_inpath = "Resources/final_activities.csv"
chat_inpath = "Resources/chatgpt_activities.csv"
activity_inpath = "Resources/activity_list.csv"
# flags_inpath = "../Resources/??????.csv"

In [91]:
# Import Files
locations_raw_df = pd.read_csv(locations_inpath,index_col=[0])
cost_analysis_df = pd.read_csv(cost_analysis_inpath,index_col=[0])
weather_df = pd.read_csv(weather_inpath,index_col=[0])
# yelp_df = pd.read_csv(yelp_inpath,index_col=[0])
chat_df = pd.read_csv(chat_inpath,index_col=[0])
activity_df = pd.read_csv(activity_inpath)
#flags_df = pd.read_csv(flags_inpath,index_col=[0])

### Clean Datasets Before Merging

In [92]:
locations_raw_df.info()
locations_raw_df["Country"] = locations_raw_df["Country"].str.replace('\u200b',"")
locations_raw_df["Country"] = locations_raw_df["Country"].str.replace('Kingdom of the Netherlands','Netherlands')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220 entries, 0 to 236
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   City            220 non-null    object 
 1   State/Province  75 non-null     object 
 2   Country         220 non-null    object 
 3   Latitude        220 non-null    float64
 4   Longitude       220 non-null    float64
dtypes: float64(2), object(3)
memory usage: 10.3+ KB


In [94]:
# cost_analysis_df.sort_values(by="cost_rank")

In [95]:
# weather_df.info()

In [96]:
# yelp_df.info()

In [97]:
df_list = [locations_raw_df,cost_analysis_df,weather_df,chat_df]  ##add yelp_df and flags_df

for i in range(len(df_list)): 
    df_list[i].columns = df_list[i].columns.str.lower()
    for x in range(len(df_list[i].columns)):
        df = df_list[i]
        col = df.columns[x]
        try:
            df[col] = df[col].str.title()
            df[col] = df[col].str.strip()
            df[col] = df[col].astype(str)
        except:
            df[col] = df[col].round()
            pass


cost_analysis_df = cost_analysis_df.rename(columns={"total_rank":"cost_rank"})
cost_analysis_df = cost_analysis_df[["city","state/province","country","latitude","longitude","cost_rank"]]
weather_df = weather_df.rename(columns={"average temperature":"avg_temp","average no sunny days":"avg_num_sun_days"})

## Merge All Dataframes Together

In [98]:
# Merge Locations onto Cost Analysis (step1_df)
step1_df = pd.merge(locations_raw_df,cost_analysis_df,how='inner',on=["city","country","latitude","longitude","state/province"])
step1_df = step1_df.drop_duplicates(subset=["city","country","latitude","longitude"])
step1_df.sort_values(by=["cost_rank"])

,city,state/province,country,latitude,longitude,cost_rank
109,Siem Reap,nan,Cambodia,13.0,104.0,1.0
99,São Paulo,nan,Brazil,-24.0,-47.0,1.0
98,Jakarta,nan,Indonesia,-6.0,107.0,1.0
96,Chiang Mai,nan,Thailand,19.0,99.0,1.0
94,Bucharest,nan,Romania,44.0,26.0,1.0
...,...,...,...,...,...,...
121,Berlin,nan,Germany,53.0,13.0,5.0
126,New York,nan,United States,41.0,-74.0,5.0
171,Brussels,nan,Belgium,51.0,4.0,5.0
201,Rotterdam,nan,Netherlands,52.0,4.0,5.0


In [72]:
# Merge (step1_df) onto Weather (step2_df)
step2_df = pd.merge(step1_df,weather_df,how='inner',on=["latitude","longitude"])
step2_df = step2_df.drop_duplicates(subset=["city","country","latitude","longitude"],keep='first')
step2_df.sort_values(by="cost_rank")

,city,state/province,country,latitude,longitude,cost_rank,month,avg_temp,avg_num_sun_days
87,St. Petersburg,Florida,Russia,60.0,30.0,1.0,3,-2.0,7.0
120,Hoi An,Nan,Vietnam,16.0,108.0,1.0,3,23.0,9.0
182,Manila,Nan,Philippines,15.0,121.0,1.0,3,27.0,12.0
37,Marrakech,Nan,Morocco,32.0,-8.0,1.0,3,21.0,11.0
117,Kyiv,Nan,Ukraine,50.0,31.0,1.0,3,5.0,9.0
...,...,...,...,...,...,...,...,...,...
125,Berlin,Nan,Germany,53.0,13.0,5.0,3,6.0,8.0
172,Brussels,Nan,Belgium,51.0,4.0,5.0,3,7.0,8.0
129,New York,Nan,United States,41.0,-74.0,5.0,3,6.0,9.0
203,Rotterdam,Nan,Netherlands,52.0,4.0,5.0,3,7.0,7.0


In [78]:
# Merge (step2_df) onto Chat (step3_df)
step3_df = pd.merge(step2_df,chat_df,how='left',on=['city','country'])
step3_df.sort_values(by="cost_rank")

,city,state/province,country,latitude,longitude,cost_rank,month,avg_temp,avg_num_sun_days,act1,act2,act3,act4,act5
71,St. Petersburg,Florida,Russia,60.0,30.0,1.0,3,-2.0,7.0,Tours,Museums,Hiking,Boat Tours,Spas
104,Hoi An,Nan,Vietnam,16.0,108.0,1.0,3,23.0,9.0,Beaches,Tours,Spas,Snorkeling,Sailing
164,Manila,Nan,Philippines,15.0,121.0,1.0,3,27.0,12.0,NaN,NaN,NaN,NaN,NaN
30,Marrakech,Nan,Morocco,32.0,-8.0,1.0,3,21.0,11.0,NaN,NaN,NaN,NaN,NaN
101,Kyiv,Nan,Ukraine,50.0,31.0,1.0,3,5.0,9.0,Tours,Spas,Massage,Hot Springs,Hiking
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,Berlin,Nan,Germany,53.0,13.0,5.0,3,6.0,8.0,NaN,NaN,NaN,NaN,NaN
154,Brussels,Nan,Belgium,51.0,4.0,5.0,3,7.0,8.0,NaN,NaN,NaN,NaN,NaN
113,New York,Nan,United States,41.0,-74.0,5.0,3,6.0,9.0,NaN,NaN,NaN,NaN,NaN
183,Rotterdam,Nan,Netherlands,52.0,4.0,5.0,3,7.0,7.0,NaN,NaN,NaN,NaN,NaN


In [99]:
locations_raw_df["country"].unique()

array(['United States', 'Cook Islands', 'United Kingdom', 'Australia',
       'Canada', 'Croatia', 'Antigua And Barbuda', 'United Arab Emirates',
       'Turks And Caicos Islands', 'Spain', 'South Africa', 'New Zealand',
       'Tanzania', 'Mauritius', 'Italy', 'Cayman Islands', 'Seychelles',
       'Morocco', 'France', 'India', 'Barbados', 'Mexico', 'Norway',
       'Jamaica', 'Netherlands', 'Denmark', 'Fiji', 'Russia',
       'Saint Lucia', 'Bahamas', 'Switzerland', 'Indonesia', 'Thailand',
       'Portugal', 'Argentina', 'Hungary', 'Turkey', 'Romania', 'Brazil',
       'Bulgaria', 'Vietnam', 'Poland', 'Malaysia', 'Singapore', 'Serbia',
       'Czech Republic', 'Cambodia', 'Ukraine', 'Sri Lanka',
       'South Korea', 'Chile', 'Germany', 'Japan', 'Austria', 'Colombia',
       'Nepal', 'Taiwan', 'Greece', 'China', 'Peru', 'Belarus',
       'Slovenia', 'Ecuador', 'Estonia', 'Uruguay', 'Belgium', 'Ireland',
       'Latvia', 'Lithuania', 'Sweden', 'Philippines', 'Costa Rica',
       'Fin

In [14]:
# Merge (step3_df) onto Yelp (step4_df)

In [15]:
# Merge (step4_df) onto Flags (final_merged_df)

In [ ]:
# Clean final DF Activities to be one of the following list: 
#'beaches','hanggliding','horsebackriding', 'hiking','hot_air_balloons',
#'paddleboarding','parasailing','sailing','snorkeling','ziplining', 
#'Spas','hotsprings','massage','tours'

## Separate Out into Tables for SQL Import

### List of all SQL Tables and Columns

#### Locations
    LocationID(pk), City, Locality, Country, Latitude, Longitude

#### LocationsCharacteristics
    LocationID(pk), LodgingScore, TransportationScore, RestaurantScore

#### Temperature
    LocationID(pk), January, February, March, April, May, June, July, August, September, October, November, December

#### SunHours
    LocationID(pk), January, February, March, April, May, June, July, August, September, October, November, December

#### Activities
    LocationID(pk),  Activity1, Activity2, Activity3, Activity4, Activity5

#### ActivityList
    ActivityID(pk), Name, Image, Attribution, Link

In [ ]:
# add ActivityID

#### CountryFlags
    Country(pk), Image

In [ ]:
## THIS ONE DOES NOT NEED TO BE HERE

## Export All Data Into SQL Tables

In [16]:
# Create outpaths based on ERD
locations_outpath = "../Resources/SQL_tables/locations.csv"
locationCharacteristics_outpath = "../Resources/SQL_tables/locationCharacteristics.csv"
countryFlags_outpath = "../Resources/SQL_tables/countryFlags.csv"
temperature_outpath = "../Resources/SQL_tables/temperature.csv"
sunHours_outpath = "../Resources/SQL_tables/sunHours.csv"
activities_outpath = "../Resources/SQL_tables/activities.csv"
activityList_outpath = "../Resources/SQL_tables/activityList.csv"

In [17]:
# Export all dataframes
locations_df.to_csv(locations_outpath)
locationCharacteristics_df.to_csv(locationCharacteristics_outpath)
countryFlags_df.to_csv(countryFlags_outpath)
temperature_df.to_csv(temperature_outpath)
sunHours_df.to_csv(sunHours_outpath)
activities_df.to_csv(activities_outpath)
activityList_df.to_csv(activityList_outpath)

NameError: name 'locations_df' is not defined